In [2]:
!pip install tensorflow_datasets

In [6]:
# import libraries
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, SimpleRNN, Bidirectional, LSTM, BatchNormalization
from keras.datasets import imdb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow as tf
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import GridSearchCV



Download and Preprocess the Dataset.

In [7]:
# Download the plain text dataset
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteW2QE7C/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteW2QE7C/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteW2QE7C/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [8]:
# Get the train and test sets
train_data, test_data = imdb['train'], imdb['test']

# Initialize sentences and labels lists
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# Loop over all training examples and save the sentences and labels
for s,l in train_data:
    training_sentences.append(s.numpy().decode('utf8'))
    training_labels.append(l.numpy())

# Loop over all test examples and save the sentences and labels
for s,l in test_data:
    testing_sentences.append(s.numpy().decode('utf8'))
    testing_labels.append(l.numpy())

# Convert labels lists to numpy array
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [ ]:
testing_sentences

In [9]:
# Parameters
vocab_size = 10000
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"
NUM_EPOCHS = 3
BATCH_SIZE = 128

# Initialize the Tokenizer class
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)

# Generate the word index dictionary for the training sentences
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

# Generate and pad the training sequences
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

# Generate and pad the test sequences
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)

Before establishing the models, it's important to create the following function to facilitate the visualization of accuracy and loss trends post-training.

In [10]:
# Plot Utility
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [11]:
# For saving history
def save_history(history, name):
    df_history = pd.DataFrame()
    for key in history.history.keys():
        df_history[key] = history.history[key]

    df_history.to_csv(f'{name}.csv')

history_test = []

"Model 1: Flattening Layer
Our initial approach involves incorporating a Flatten layer right after the embedding layer. One of its primary advantages is its rapid training speed."

In [ ]:
# Parameters
embedding_dim = 16
dense_dim = 6

# Model Definition with a Flatten layer
model_flatten = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Set the training parameters
model_flatten.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Print the model summary
model_flatten.summary()

In [ ]:
# NUM_EPOCHS = 10
# BATCH_SIZE = 128

# Train the model
history_flatten = model_flatten.fit(padded,
                                    training_labels_final,
                                    batch_size=BATCH_SIZE,
                                    epochs=NUM_EPOCHS,
                                    validation_data=(testing_padded, testing_labels_final))

In [ ]:
# Plot the accuracy and loss history
plot_graphs(history_flatten, 'accuracy')
plot_graphs(history_flatten, 'loss')

In [ ]:
predictions = model_flatten.predict(testing_padded)

# To get a clearer view of forecasts, it can be created in binary format:
binary_predictions = ['positive' if pred > 0.5 else 'negative' for pred in predictions]

# Some predicted values can be displayed alongside the actual values for comparison
for i in range(10):
    print(f"Predicted: {binary_predictions[i]}, Label: {'positive' if testing_labels_final[i] > 0.5 else 'negative'}")

In [ ]:
# Model evaluation
# This will give the overall accuracy of the model on the test data
test_loss, test_acc = model_flatten.evaluate(testing_padded, testing_labels_final)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

In [ ]:
# saving results
save_history(history_flatten,'flatten')
history_test.append(('FLATTEN', test_loss, test_acc))

"Model 2: Long Short-Term Memory (LSTM)
The next model employs LSTM, which might have a slower training pace but proves valuable in scenarios where token order holds significance."

In [ ]:
# Parameters
embedding_dim = 16
lstm_dim = 32
dense_dim = 6

# Model Definition with LSTM
model_lstm = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_dim)),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Set the training parameters
model_lstm.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Print the model summary
model_lstm.summary()

In [ ]:
# NUM_EPOCHS = 10
# BATCH_SIZE = 128

# Train the model
history_lstm = model_lstm.fit(padded,
                              training_labels_final,
                              batch_size=BATCH_SIZE,
                              epochs=NUM_EPOCHS,
                              validation_data=(testing_padded, testing_labels_final))

In [ ]:
# Plot the accuracy and loss history
plot_graphs(history_lstm, 'accuracy')
plot_graphs(history_lstm, 'loss')


In [ ]:
predictions = model_lstm.predict(testing_padded)

# To get a clearer view of forecasts, it can be created in binary format:
binary_predictions = ['positive' if pred > 0.5 else 'negative' for pred in predictions]

# Some predicted values can be displayed alongside the actual values for comparison
for i in range(10):
    print(f"Predicted: {binary_predictions[i]}, Label: {'positive' if testing_labels_final[i] > 0.5 else 'negative'}")

In [ ]:
# Model evaluation
# This will give the overall accuracy of the model on the test data
test_loss, test_acc = model_lstm.evaluate(testing_padded, testing_labels_final)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

In [ ]:
# saving results
save_history(history_lstm,'lstm')
history_test.append(('LSTM', test_loss, test_acc))

"Model 3: Gated Recurrent Unit (GRU)
The Gated Recurrent Unit, or GRU, is often regarded as a more straightforward alternative to LSTM. It finds its place in applications where sequence matters, but expedited results are desired, even if a slight trade-off in accuracy can be accepted."

In [ ]:
# Parameters
embedding_dim = 16
gru_dim = 32
dense_dim = 6

# Model Definition with GRU
model_gru = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(gru_dim)),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Set the training parameters
model_gru.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Print the model summary
model_gru.summary()

In [ ]:
# NUM_EPOCHS = 10
# BATCH_SIZE = 128

# Train the model
history_gru = model_gru.fit(padded,
                            training_labels_final,
                            batch_size=BATCH_SIZE,
                            epochs=NUM_EPOCHS,
                            validation_data=(testing_padded, testing_labels_final))

In [ ]:
# Plot the accuracy and loss history
plot_graphs(history_gru, 'accuracy')
plot_graphs(history_gru, 'loss')

In [ ]:
predictions = model_gru.predict(testing_padded)

# To get a clearer view of forecasts, it can be created in binary format:
binary_predictions = ['positive' if pred > 0.5 else 'negative' for pred in predictions]

# Some predicted values can be displayed alongside the actual values for comparison
for i in range(10):
    print(f"Predicted: {binary_predictions[i]}, Label: {'positive' if testing_labels_final[i] > 0.5 else 'negative'}")

In [ ]:
# Model evaluation
# This will give the overall accuracy of the model on the test data
test_loss, test_acc = model_gru.evaluate(testing_padded, testing_labels_final)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

In [ ]:
# saving results
save_history(history_gru,'gru')
history_test.append(('GRU', test_loss, test_acc))

"Model 4: Convolutional Approach
Similar to the Flatten-based model, this approach also offers significantly faster training compared to models incorporating RNN layers such as LSTM and GRU."

In [ ]:
# Parameters
embedding_dim = 16
filters = 128
kernel_size = 5
dense_dim = 6

# Model Definition with Conv1D
model_conv = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Conv1D(filters, kernel_size, activation='relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(dense_dim, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Set the training parameters
model_conv.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Print the model summary
model_conv.summary()

In [ ]:
# NUM_EPOCHS = 10
# BATCH_SIZE = 128

# Train the model
history_conv = model_conv.fit(padded,
                              training_labels_final,
                              batch_size=BATCH_SIZE,
                              epochs=NUM_EPOCHS,
                              validation_data=(testing_padded, testing_labels_final))

In [ ]:
# Plot the accuracy and loss history
plot_graphs(history_conv, 'accuracy')
plot_graphs(history_conv, 'loss')

In [ ]:
predictions = model_conv.predict(testing_padded)

# To get a clearer view of forecasts, it can be created in binary format:
binary_predictions = ['positive' if pred > 0.5 else 'negative' for pred in predictions]

# Some predicted values can be displayed alongside the actual values for comparison
for i in range(10):
    print(f"Predicted: {binary_predictions[i]}, Label: {'positive' if testing_labels_final[i] > 0.5 else 'negative'}")

In [ ]:
# Model evaluation
# This will give the overall accuracy of the model on the test data
test_loss, test_acc = model_conv.evaluate(testing_padded, testing_labels_final)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

In [ ]:
# saving results
save_history(history_conv,'convolution')
history_test.append(('CONVOLUTION', test_loss, test_acc))

Model 5: SRNN

In [ ]:
# Parameters
max_features = 20000
maxlen = 120  # cut texts after this number of words (among top max_features most common words)
embedding_vector_length = 32

# Model Definition
model_srnn = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=max_features, output_dim=embedding_vector_length, input_length=maxlen),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.SimpleRNN(32),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

# Set the training parameters
model_srnn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Print the model summary
model_srnn.summary()

In [ ]:
# NUM_EPOCHS = 10
# BATCH_SIZE = 128

# Train the model
history_srnn = model_srnn.fit(padded,
                              training_labels_final,
                              batch_size=BATCH_SIZE,
                              epochs=NUM_EPOCHS,
                              validation_data=(testing_padded, testing_labels_final))

In [ ]:
# Plot the accuracy and loss history
plot_graphs(history_srnn, 'accuracy')
plot_graphs(history_srnn, 'loss')

In [ ]:
predictions = model_srnn.predict(testing_padded)

# To get a clearer view of forecasts, it can be created in binary format:
binary_predictions = ['positive' if pred > 0.5 else 'negative' for pred in predictions]

# Some predicted values can be displayed alongside the actual values for comparison
for i in range(10):
    print(f"Predicted: {binary_predictions[i]}, Label: {'positive' if testing_labels_final[i] > 0.5 else 'negative'}")

In [ ]:
# Model evaluation
# This will give the overall accuracy of the model on the test data
test_loss, test_acc = model_srnn.evaluate(testing_padded, testing_labels_final)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

In [ ]:
# saving results
save_history(history_srnn,'srnn')
history_test.append(('SRNN', test_loss, test_acc))

Model 6: BRNN

In [ ]:
# Parameters
max_features = 20000
maxlen = 120  # cut texts after this number of words (among top max_features most common words)
embedding_vector_length = 32

# Model Definition
model_brnn = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=max_features, output_dim=embedding_vector_length, input_length=maxlen),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Bidirectional(LSTM(32)),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

# Set the training parameters
model_brnn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Print the model summary
model_brnn.summary()

In [ ]:
# NUM_EPOCHS = 10
# BATCH_SIZE = 128

# Train the model
history_brnn = model_brnn.fit(padded,
                              training_labels_final,
                              batch_size=BATCH_SIZE,
                              epochs=NUM_EPOCHS,
                              validation_data=(testing_padded, testing_labels_final))

In [ ]:
# Plot the accuracy and loss history
plot_graphs(history_brnn, 'accuracy')
plot_graphs(history_brnn, 'loss')

In [ ]:
predictions = model_brnn.predict(testing_padded)

# To get a clearer view of forecasts, it can be created in binary format:
binary_predictions = ['positive' if pred > 0.5 else 'negative' for pred in predictions]

# Some predicted values can be displayed alongside the actual values for comparison

for i in range(10):
    print(f"Predicted: {binary_predictions[i]}, Label: {'positive' if testing_labels_final[i] > 0.5 else 'negative'}")


In [ ]:
# Model evaluation
# This will give the overall accuracy of the model on the test data
test_loss, test_acc = model_brnn.evaluate(testing_padded, testing_labels_final)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

In [ ]:
# saving results
save_history(history_brnn,'brnn')
history_test.append(('BRNN', test_loss, test_acc))

Model 7: Deep BRNN

In [ ]:
# Parameters
max_features = 20000
maxlen = 120  # cut texts after this number of words (among top max_features most common words)
embedding_vector_length = 32

# Model Definition
model_dbrnn = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=max_features, output_dim=embedding_vector_length, input_length=maxlen),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Bidirectional(LSTM(32, return_sequences=True)),
    tf.keras.layers.Bidirectional(LSTM(32)),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])

# Set the training parameters
model_dbrnn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

# Print the model summary
model_dbrnn.summary()

In [ ]:
# NUM_EPOCHS = 10
# BATCH_SIZE = 128
"HARDCODED_EPOCHS = 5"
# Train the model
history_dbrnn = model_dbrnn.fit(padded,
                              training_labels_final,
                              batch_size=BATCH_SIZE,
                              epochs=5,
                              validation_data=(testing_padded, testing_labels_final))

In [ ]:
# Plot the accuracy and loss history
plot_graphs(history_dbrnn, 'accuracy')
plot_graphs(history_dbrnn, 'loss')

In [ ]:
predictions = model_dbrnn.predict(testing_padded)

# To get a clearer view of forecasts, it can be created in binary format:
binary_predictions = ['positive' if pred > 0.5 else 'negative' for pred in predictions]

# Some predicted values can be displayed alongside the actual values for comparison
for i in range(10):
    print(f"Predicted: {binary_predictions[i]}, Label: {'positive' if testing_labels_final[i] > 0.5 else 'negative'}")

In [ ]:
# Model evaluation
# This will give the overall accuracy of the model on the test data
test_loss, test_acc = model_dbrnn.evaluate(testing_padded, testing_labels_final)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

In [ ]:
# saving results
save_history(history_dbrnn,'dbrnn')
history_test.append(('DBRNN', test_loss, test_acc))

Summing up:


In [ ]:
csv_s = ['flatten', 'lstm', 'gru', 'convolution', 'srnn','brnn', 'dbrnn']
headers = ['Model', 'Loss', 'Accuracy']

In [ ]:
data = []
for m_name in csv_s:
    df_history = pd.read_csv(m_name+'.csv')
    loss, acc = df_history['loss'][2], df_history['accuracy'][2]
    data.append((m_name.upper(), loss, acc))

In [ ]:
from tabulate import tabulate
print(tabulate(data, headers=headers))

In [ ]:
print(tabulate(history_test, headers=headers))

Conclusion: All the neural networks employed in the study exhibited relatively high and comparable performance. A careful analysis of the training process revealed that, starting from the 3rd epoch, all neural networks began to exhibit signs of overfitting. Consequently, the training duration was constrained to a maximum of three epochs. Among the models, the "Flatten" neural network emerged as the standout performer, boasting the lowest validation loss and a commendable level of accuracy.

EXPEREMENTAL
<(-_=)>


In [ ]:
max_features = 20000
maxlen = 120
embedding_vector_length = 32

In [ ]:
# model = Sequential()
# model.add(Embedding(input_dim=max_features, output_dim=embedding_vector_length, input_length=maxlen))
# model.add(Dropout(0.2))
# model.add(Bidirectional(LSTM(64, return_sequences=True, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))))
# model.add(Bidirectional(LSTM(64, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))))
# model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))
# model.add(Dense(1, activation='tanh'))


# Define a function to build the model for hyperparameter tuning
# def build_model(hidden_units=64, dropout_rate=0.2, learning_rate=0.001, use_batch_norm=True):
#     model = Sequential()
#     model.add(Embedding(input_dim=max_features, output_dim=embedding_vector_length, input_length=maxlen))
#     model.add(Dropout(dropout_rate))
#     model.add(Bidirectional(LSTM(hidden_units, return_sequences=True, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))))
#     model.add(Bidirectional(LSTM(hidden_units, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))))

#     if use_batch_norm:
#         model.add(BatchNormalization())

#     model.add(Dropout(dropout_rate))
#     model.add(Dense(1, activation='sigmoid'))

#     optimizer = Adam(learning_rate=learning_rate)
#     model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

#     return model

# # Hyperparameter Tuning
# param_grid = {
#     'hidden_units': [32, 64, 128],
#     'dropout_rate': [0.2, 0.3, 0.4],
#     'learning_rate': [0.001, 0.01],
#     'use_batch_norm': [True, False]
# }

# # Use GridSearchCV to find the best combination of hyperparameters
# model = build_model()
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
# grid_result = grid.fit(padded, training_labels_final)

# # Print the best hyperparameters
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# # Build the final model with the best hyperparameters
# best_params = grid_result.best_params_
# final_model = build_model(hidden_units=best_params['hidden_units'],
#                           dropout_rate=best_params['dropout_rate'],
#                           learning_rate=best_params['learning_rate'],
#                           use_batch_norm=best_params['use_batch_norm'])

# # Train the final model
# final_model.fit(padded, training_labels_final,  validation_data=(testing_padded, testing_labels_final), epochs=30, batch_size=64, callbacks=[early_stopping, model_checkpoint])

# # Evaluate the final model on the test set
# test_loss, test_accuracy = final_model.evaluate(testing_padded, testing_labels_final)
# print(f'Test Loss: {test_loss:.4f}')
# print(f'Test Accuracy: {test_accuracy:.4f}')



# Parameters
max_features = 20000
maxlen = 120
embedding_vector_length = 32

# Define a function to build the model for hyperparameter tuning
def build_model(hidden_units=64, dropout_rate=0.2, learning_rate=0.001, use_batch_norm=True):
    model = Sequential()
    model.add(Embedding(input_dim=max_features, output_dim=embedding_vector_length, input_length=maxlen))
    model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(hidden_units, return_sequences=True, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))))
    model.add(Bidirectional(LSTM(hidden_units, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))))

    if use_batch_norm:
        model.add(BatchNormalization())

    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Hyperparameter Tuning
param_grid = {
    'hidden_units': [32, 64, 128],
    'dropout_rate': [0.2, 0.3, 0.4],
    'learning_rate': [0.001, 0.01],
    'use_batch_norm': [True, False]
}

# Use GridSearchCV to find the best combination of hyperparameters
model = build_model()
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(padded, training_labels_final)

# Print the best hyperparameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Build the final model with the best hyperparameters
best_params = grid_result.best_params_
final_model = build_model(hidden_units=best_params['hidden_units'],
                          dropout_rate=best_params['dropout_rate'],
                          learning_rate=best_params['learning_rate'],
                          use_batch_norm=best_params['use_batch_norm'])

# Train the final model
final_model.fit(padded, training_labels_final, validation_data=(X_test, y_test), epochs=30, batch_size=64, callbacks=[early_stopping, model_checkpoint])

# Evaluate the final model on the test set
test_loss, test_accuracy = final_model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


"""

I defined a function build_model that allows to vary hyperparameters such as the number of hidden units, dropout rate, learning rate, and whether or not to use batch normalization.

Hyperparameter tuning is performed using GridSearchCV to search for the best combination of hyperparameters from the defined parameter grid.

The best hyperparameters are selected, and the final model is built with these parameters.

The final model is trained, and its performance is evaluated on the test set.

By using GridSearchCV, you can systematically search for the best hyperparameters for your model, and the final model is trained with the optimal configuration. This approach helps you find the best trade-off between model complexity and generalization.

"""





In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import accuracy_score

# Parameters
max_features = 20000
maxlen = 120
embedding_vector_length = 32

# Define a function to build the model
def build_model(hidden_units=64, dropout_rate=0.2, learning_rate=0.001, use_batch_norm=True):
    model = Sequential()
    model.add(Embedding(input_dim=max_features, output_dim=embedding_vector_length, input_length=maxlen))
    model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(hidden_units, return_sequences=True, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))))
    model.add(Bidirectional(LSTM(hidden_units, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))))

    if use_batch_norm:
        model.add(BatchNormalization())

    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Define a custom accuracy function
def custom_accuracy(y_true, y_pred):
    y_pred_binary = [1 if pred >= 0.5 else 0 for pred in y_pred]
    return accuracy_score(y_true, y_pred_binary)

# Define hyperparameters
hidden_units = 64
dropout_rate = 0.2
learning_rate = 0.001
use_batch_norm = True

# Build the model
final_model = build_model(hidden_units, dropout_rate, learning_rate, use_batch_norm)

# Training
final_model.fit(padded, training_labels_final, validation_data=(testing_padded, testing_labels_final), epochs=30, batch_size=64, callbacks=[EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True), ModelCheckpoint('best_model.h5', save_best_only=True)])

# Evaluate the final model on the test set
test_loss, test_accuracy = final_model.evaluate(testing_padded, testing_labels_final, verbose=0)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


Epoch 1/30
102/391 [======>.......................] - ETA: 49s - loss: 3.2191 - accuracy: 0.6619

KeyboardInterrupt: ignored

In [ ]:
history = final_model.fit(padded, training_labels_final,  validation_data=(testing_padded, testing_labels_final), epochs=30, batch_size=128)


In [ ]:
# Plot the accuracy and loss history
plot_graphs(history, 'accuracy')
plot_graphs(history, 'loss')

In [ ]:
# Plot training history
plt.plot(history.history['accuracy'], label='train_accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [ ]:

# Model evaluation
# This will give the overall accuracy of the model on the test data
test_loss, test_acc = final_model.evaluate(testing_padded, testing_labels_final)
print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

The test results indicate that the model achieved a test loss of approximately 0.428 and a test accuracy of approximately 82.16%. This suggests that the model is performing fairly well on the test data, correctly classifying instances with an accuracy of around 82.16%. These values can be considered quite good, but the interpretation may vary depending on the specific problem and domain. It's also worth noting that further fine-tuning and experimentation can potentially improve these metrics.

In [ ]:
# Make predictions
predictions = final_model.predict(testing_padded)

# To get a clearer view of forecasts, it can be created in binary format:
binary_predictions = ['positive' if pred > 0.5 else 'negative' for pred in predictions]

# Some predicted values can be displayed alongside the actual values for comparison
for i in range(10):
    print(f"Predicted: {binary_predictions[i]}, Label: {'positive' if testing_labels_final[i] > 0.5 else 'negative'}")



In [ ]:
# saving results
save_history(history,'custom_model')
history_test.append(('custom_model', test_loss, test_acc))

In [ ]:
csv_s = ['flatten', 'lstm', 'gru', 'convolution', 'srnn', 'brnn', 'dbrnn', 'custom_model']
headers = ['Model', 'Loss', 'Accuracy']

In [ ]:
data = []
for m_name in csv_s:
    df_history = pd.read_csv(m_name+'.csv')
    loss, acc = df_history['loss'][2], df_history['accuracy'][2]
    data.append((m_name.upper(), loss, acc))

In [ ]:
from tabulate import tabulate
print(tabulate(data, headers=headers))

In [ ]:
print(tabulate(history_test, headers=headers))

In [ ]:
!pip install keras==2.12.0

In [ ]:
!pip install scikeras


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import accuracy_score
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

# Parameters
max_features = 20000
maxlen = 120
embedding_vector_length = 32

# Define a function to build the model
def build_model(hidden_units=64, dropout_rate=0.2, learning_rate=0.001, use_batch_norm=True):
    model = Sequential()
    model.add(Embedding(input_dim=max_features, output_dim=embedding_vector_length, input_length=maxlen))
    model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(hidden_units, return_sequences=True, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))))
    model.add(Bidirectional(LSTM(hidden_units, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))))

    if use_batch_norm:
        model.add(BatchNormalization())

    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Define a custom accuracy function
def custom_accuracy(y_true, y_pred):
    y_pred_binary = [1 if pred >= 0.5 else 0 for pred in y_pred]
    return accuracy_score(y_true, y_pred_binary)

# Hyperparameter tuning using grid search
param_grid = {
    'hidden_units': [64, 128],
    'dropout_rate': [0.2, 0.4],
    'learning_rate': [0.001, 0.01],
    'use_batch_norm': [True, False]
}

scorer = make_scorer(custom_accuracy)

# Define a custom wrapper class
class CustomKerasClassifier(KerasClassifier):
    def fit(self, X, y, **kwargs):
        return super().fit(X, y, **kwargs)

model = CustomKerasClassifier(build_fn=build_model, epochs=30, batch_size=64, verbose=0)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, scoring=scorer)
grid_result = grid.fit(padded, training_labels_final)

# Print the best hyperparameters
print("Best Hyperparameters: ", grid_result.best_params_)

# Build the final model with the best hyperparameters
best_params = grid_result.best_params_
final_model = build_model(
    hidden_units=best_params['hidden_units'],
    dropout_rate=best_params['dropout_rate'],
    learning_rate=best_params['learning_rate'],
    use_batch_norm=best_params['use_batch_norm']
)

# Training
final_model.fit(padded, training_labels_final, validation_data=(testing_padded, testing_labels_final), epochs=30, batch_size=64, callbacks=[EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True), ModelCheckpoint('best_model.h5', save_best_only=True)])

# Evaluate the final model on the test set
test_loss, test_accuracy = final_model.evaluate(testing_padded, testing_labels_final, verbose=0)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

# Parameters
max_features = 20000
maxlen = 120
embedding_vector_length = 32

# Define a function to build the model
def build_model(hidden_units=64, dropout_rate=0.2, learning_rate=0.001, use_batch_norm=True):
    model = Sequential()
    model.add(Embedding(input_dim=max_features, output_dim=embedding_vector_length, input_length=maxlen))
    model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(hidden_units, return_sequences=True, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))))
    model.add(Bidirectional(LSTM(hidden_units, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01))))

    if use_batch_norm:
        model.add(BatchNormalization())

    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Define a custom accuracy function
def custom_accuracy(y_true, y_pred):
    y_pred_binary = [1 if pred >= 0.5 else 0 for pred in y_pred]
    return accuracy_score(y_true, y_pred_binary)

# Define hyperparameters
param_grid = {
    'hidden_units': [64, 128],
    'dropout_rate': [0.2, 0.4],
    'learning_rate': [0.001, 0.01],
    'use_batch_norm': [True, False]
}

# Create a custom scorer using the custom accuracy function
scorer = make_scorer(custom_accuracy)

best_accuracy = 0
best_params = {}

# Perform a grid search for hyperparameter tuning
for hidden_units in param_grid['hidden_units']:
    for dropout_rate in param_grid['dropout_rate']:
        for learning_rate in param_grid['learning_rate']:
            for use_batch_norm in param_grid['use_batch_norm']:
                print(f"Training model with hidden_units={hidden_units}, dropout_rate={dropout_rate}, learning_rate={learning_rate}, use_batch_norm={use_batch_norm}")

                # Build and train the model
                model = build_model(hidden_units, dropout_rate, learning_rate, use_batch_norm)
                model.fit(padded, training_labels_final, validation_data=(testing_padded, testing_labels_final), epochs=30, batch_size=64, callbacks=[EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True), ModelCheckpoint('best_model.h5', save_best_only=True)])

                # Evaluate the model
                _, test_accuracy = model.evaluate(testing_padded, testing_labels_final, verbose=0)
                if test_accuracy > best_accuracy:
                    best_accuracy = test_accuracy
                    best_params = {
                        'hidden_units': hidden_units,
                        'dropout_rate': dropout_rate,
                        'learning_rate': learning_rate,
                        'use_batch_norm': use_batch_norm
                    }

# Print the best hyperparameters and accuracy
print("Best Hyperparameters: ", best_params)
print("Best Test Accuracy: ", best_accuracy)


Training model with hidden_units=64, dropout_rate=0.2, learning_rate=0.001, use_batch_norm=True


NameError: ignored